# Bonds #
## 25. Mischievous pricing conventions ##

In [1]:
import QuantLib as ql
import pandas as pd

In [7]:
today = ql.Date(5, ql.January, 2010)
ql.Settings.instance().evaluationDate = today
discounting_curve = ql.RelinkableYieldTermStructureHandle()
forecasting_curve = ql.RelinkableYieldTermStructureHandle()
index = ql.USDLibor(ql.Period(3, ql.Months), forecasting_curve)
settlement_days = 0
calendar = ql.NullCalendar()
face_amount = 100.0
schedule = ql.Schedule(today, today + ql.Period(4, ql.Years),
                       ql.Period(3, ql.Months), calendar,
                       ql.Unadjusted, ql.Unadjusted,
                       ql.DateGeneration.Forward, False)
bond = ql.FloatingRateBond(settlement_days, face_amount, schedule,
                           index, ql.Thirty360(ql.Thirty360.BondBasis), ql.Unadjusted, fixingDays = 0)
bond.setPricingEngine(ql.DiscountingBondEngine(discounting_curve))

In [8]:
flat_rate = ql.FlatForward(today, 0.10, ql.Thirty360(ql.Thirty360.BondBasis), ql.Compounded, ql.Quarterly)
forecasting_curve.linkTo(flat_rate)
discounting_curve.linkTo(flat_rate)

In [9]:
print(bond.cleanPrice())

99.5433545426823


## What is happening here? ##

In [10]:
print(flat_rate.dayCounter())
print(ql.as_coupon(bond.cashflows()[0]).dayCounter())
print(index.dayCounter())

30/360 (Bond Basis) day counter
30/360 (Bond Basis) day counter
Actual/360 day counter


In [11]:
coupons = [ql.as_coupon(c) for c in bond.cashflows()[:-1]]
pd.DataFrame([(c.date(), c.rate(), c.accrualPeriod()) for c in coupons ],
             columns=('Date', 'Rate', 'Accrual period'),
             index=range(1,len(coupons)+1))

,Date,Rate,Accrual period
1,"April 5th, 2010",0.100014,0.25
2,"July 5th, 2010",0.098901,0.25
3,"October 5th, 2010",0.097826,0.25
4,"January 5th, 2011",0.097826,0.25
5,"April 5th, 2011",0.100000,0.25
6,"July 5th, 2011",0.098901,0.25
7,"October 5th, 2011",0.097826,0.25
8,"January 5th, 2012",0.097899,0.25
9,"April 5th, 2012",0.098952,0.25
10,"July 5th, 2012",0.098849,0.25


### The importance of being consistent ###

In [12]:
bond = ql.FloatingRateBond(settlement_days, face_amount, schedule, index, 
                           ql.Actual360(), ql.Unadjusted, fixingDays = 0)
bond.setPricingEngine(ql.DiscountingBondEngine(discounting_curve))

In [13]:
flat_rate_2 = ql.FlatForward(today, 0.10, ql.Actual360(), ql.Compounded, ql.Quarterly)
forecasting_curve.linkTo(flat_rate_2)
discounting_curve.linkTo(flat_rate_2)

In [14]:
print(bond.cleanPrice())

100.00117521248728


In [15]:
coupons = [ql.as_coupon(c) for c in bond.cashflows()[:-1]]
pd.DataFrame([(c.date(), c.rate(), c.accrualPeriod()) for c in coupons ],
             columns=('Date', 'Rate', 'Accrual period'),
             index=range(1,len(coupons)+1))

,Date,Rate,Accrual period
1,"April 5th, 2010",0.100014,0.250000
2,"July 5th, 2010",0.100014,0.252778
3,"October 5th, 2010",0.100028,0.255556
4,"January 5th, 2011",0.100028,0.255556
5,"April 5th, 2011",0.100000,0.250000
6,"July 5th, 2011",0.100014,0.252778
7,"October 5th, 2011",0.100028,0.255556
8,"January 5th, 2012",0.100055,0.255556
9,"April 5th, 2012",0.100041,0.252778
10,"July 5th, 2012",0.099986,0.252778


In [17]:
index = ql.IborIndex('Mock Libor', ql.Period(3, ql.Months), 0, ql.USDCurrency(),
                     ql.NullCalendar(), ql.Unadjusted, False, ql.Thirty360(ql.Thirty360.BondBasis),
                     forecasting_curve)
bond = ql.FloatingRateBond(settlement_days, face_amount, schedule, index, ql.Thirty360(ql.Thirty360.BondBasis),
                           ql.Unadjusted, fixingDays = 0)
bond.setPricingEngine(ql.DiscountingBondEngine(discounting_curve))

In [18]:
forecasting_curve.linkTo(flat_rate)
discounting_curve.linkTo(flat_rate)

In [19]:
print(bond.cleanPrice())

100.00000000000001


In [20]:
coupons = [ql.as_coupon(c) for c in bond.cashflows()[:-1]]
pd.DataFrame([(c.date(), c.rate(), c.accrualPeriod()) for c in coupons ],
             columns=('Date', 'Rate', 'Accrual period'),
             index=range(1,len(coupons)+1))

,Date,Rate,Accrual period
1,"April 5th, 2010",0.1,0.25
2,"July 5th, 2010",0.1,0.25
3,"October 5th, 2010",0.1,0.25
4,"January 5th, 2011",0.1,0.25
5,"April 5th, 2011",0.1,0.25
6,"July 5th, 2011",0.1,0.25
7,"October 5th, 2011",0.1,0.25
8,"January 5th, 2012",0.1,0.25
9,"April 5th, 2012",0.1,0.25
10,"July 5th, 2012",0.1,0.25
